<a href="https://colab.research.google.com/github/GloryTrizza/Used-Cars-ETL/blob/main/ExtractTransformLoad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Extract Transform Load (ETL) Lab**


In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## Download Files


In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2022-12-30 20:51:31--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip.5’

datasource.zip.5    100%[===================>]   4.15K  --.-KB/s    in 0s      

2022-12-30 20:51:31 (328 MB/s) - ‘datasource.zip.5’ saved [4249/4249]



## Unzip Files


In [3]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
replace dealership_data/used_car_prices1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices3.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices1.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices2.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices3.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices1.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices2.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dealership_data/used_car_prices3.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


## About the Data


The file `dealership_data` contains CSV, JSON, and XML files for used car data which contain features named `car_model`, `year_of_manufacture`, `price`, and `fuel`.


## Set Paths


In [4]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

## Extract


### 1: CSV Extract Function


In [5]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe


### 2: JSON Extract Function


In [6]:

def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

### 3: XML Extract Function


In [7]:

def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe

### 4: Extract Function

Calling the specific extract functions created above by replacing the `ADD_FUNCTION_CALL` with the proper function call.


In [8]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data

    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)

    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)

    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)

    return extracted_data

## Transform


### 5: Transform

Round the `price` columns to 2 decimal places


In [9]:
def transform(data):
        data['price'] = round(data.price, 2)
        return data


## Loading


### 6: Load


In [10]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  


## Logging


###  7: Log

Change the timestamp order to Hour-Minute-Second-Monthname-Day-Year.


In [11]:
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL Process


###  8: ETL Process

Run all functions to extract, transform, and load the data. Log all events using the `log` function. 


In [12]:
log("ETL Job Started")

log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

log("ETL Job Ended")